In [1]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Initialization: Load shared functions and simulated data 

# Load shared functions
!curl -O https://raw.githubusercontent.com/Fraud-Detection-Handbook/fraud-detection-handbook/main/Chapter_References/shared_functions.py
%run shared_functions.py

# Get simulated data from Github repository
if not os.path.exists("simulated-data-raw"):
    !git clone https://github.com/ASIF-ISI-KOLKATA/Fraud_Transaction_Detection.git
        

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 31568  100 31568    0     0   200k      0 --:--:-- --:--:-- --:--:--  200k


In [2]:
#Loading of dataset

In [3]:
DIR_INPUT='./simulated-data-raw/data/' 

BEGIN_DATE = "2018-04-01"
END_DATE = "2018-09-30"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

Load  files
Wall time: 16.2 s
1786029 transactions loaded, containing 14753 fraudulent transactions


In [4]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-01 00:00:31,596,2144,66.50,31,0,0,0,1,...,1.0,66.50,1.0,66.50,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-04-01 00:01:59,3305,7837,75.42,119,0,0,0,1,...,1.0,75.42,1.0,75.42,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-04-01 00:03:12,53,7674,15.11,192,0,0,0,1,...,1.0,15.11,1.0,15.11,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-04-01 00:04:03,1311,3615,122.18,243,0,0,0,1,...,1.0,122.18,1.0,122.18,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-04-01 00:06:20,4459,169,60.67,380,0,0,0,1,...,1.0,60.67,1.0,60.67,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#Date and time transformations

In [6]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)

In [7]:
%time transactions_df['TX_DURING_WEEKEND']=transactions_df.TX_DATETIME.apply(is_weekend)

Wall time: 13.3 s


In [8]:
def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)

In [9]:
%time transactions_df['TX_DURING_NIGHT']=transactions_df.TX_DATETIME.apply(is_night)

Wall time: 15.9 s


In [10]:
transactions_df[transactions_df.TX_TIME_DAYS>=30]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
293172,293172,2018-05-01 00:01:40,4889,3991,24.17,2592100,30,0,0,0,...,14.0,50.157143,66.0,47.167273,1.0,0.0,7.0,0.0,20.0,0.0
293173,293173,2018-05-01 00:02:43,1553,6560,72.95,2592163,30,0,0,0,...,30.0,97.974000,136.0,94.239779,3.0,0.0,12.0,0.0,26.0,0.0
293174,293174,2018-05-01 00:03:25,1545,3652,13.75,2592205,30,0,0,0,...,27.0,13.924444,92.0,13.522391,0.0,0.0,4.0,0.0,23.0,0.0
293175,293175,2018-05-01 00:03:39,2725,467,90.41,2592219,30,0,0,0,...,18.0,74.080000,64.0,74.210156,1.0,0.0,7.0,0.0,16.0,0.0
293176,293176,2018-05-01 00:04:39,2039,9217,34.36,2592279,30,0,0,0,...,14.0,29.107857,76.0,27.367632,2.0,0.0,7.0,0.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786024,1786024,2018-09-30 23:57:44,1963,9225,19.58,15811064,182,0,0,1,...,32.0,92.100938,122.0,89.237049,2.0,0.0,6.0,0.0,35.0,0.0
1786025,1786025,2018-09-30 23:58:24,2878,4178,56.47,15811104,182,0,0,1,...,23.0,52.699130,103.0,54.361942,1.0,0.0,11.0,0.0,38.0,0.0
1786026,1786026,2018-09-30 23:58:26,3630,8878,12.55,15811106,182,0,0,1,...,17.0,59.020000,67.0,59.234627,0.0,0.0,8.0,0.0,30.0,0.0
1786027,1786027,2018-09-30 23:59:31,3531,192,101.36,15811171,182,0,0,1,...,13.0,58.278462,71.0,56.860986,1.0,0.0,9.0,0.0,26.0,0.0


In [11]:
#Customer ID transformations

In [12]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Let us first order transactions chronologically
    customer_transactions=customer_transactions.sort_values('TX_DATETIME')
    
    # The transaction date and time is set as the index, which will allow the use of the rolling function 
    customer_transactions.index=customer_transactions.TX_DATETIME
    
    # For each window size
    for window_size in windows_size_in_days:
        
        # Compute the sum of the transaction amounts and the number of transactions for the given window size
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Compute the average transaction amount for the given window size
        # NB_TX_WINDOW is always >0 since current transaction is always included
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Save feature values
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Reindex according to transaction IDs
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # And return the dataframe with the new features
    return customer_transactions

In [13]:
#Let us compute these aggregates for the first customer
spending_behaviour_customer_0=get_customer_spending_behaviour_features(transactions_df[transactions_df.CUSTOMER_ID==0])
spending_behaviour_customer_0

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,,,,,
8908,8908,2018-04-01 19:24:30,0,6667,2.90,69870,0,0,0,1,...,1.0,2.900000,1.0,2.900000,0.0,0.0,0.0,0.0,0.0,0.0
16734,16734,2018-04-02 15:13:02,0,4870,4.56,141182,1,0,0,0,...,2.0,3.730000,2.0,3.730000,0.0,0.0,0.0,0.0,0.0,0.0
18171,18171,2018-04-02 18:00:16,0,905,6.22,151216,1,0,0,0,...,3.0,4.560000,3.0,4.560000,0.0,0.0,0.0,0.0,0.0,0.0
19320,19320,2018-04-02 22:56:38,0,7192,11.39,168998,1,0,0,0,...,4.0,6.267500,4.0,6.267500,0.0,0.0,0.0,0.0,0.0,0.0
28701,28701,2018-04-03 19:50:04,0,8755,2.60,244204,2,0,0,0,...,5.0,5.534000,5.0,5.534000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754304,1754304,2018-09-27 16:11:47,0,93,7.18,15523907,179,0,0,0,...,9.0,4.368889,34.0,4.794118,0.0,0.0,10.0,0.0,30.0,0.0
1771164,1771164,2018-09-29 11:48:04,0,6960,10.98,15680884,181,0,0,1,...,8.0,5.038750,34.0,4.914412,3.0,0.0,12.0,0.0,41.0,0.0
1776229,1776229,2018-09-29 23:45:35,0,8025,0.97,15723935,181,0,0,1,...,6.0,5.236667,32.0,4.876875,2.0,0.0,10.0,0.0,34.0,0.0


In [14]:
#Let us now generate these features for all customers
%time transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

Wall time: 44.3 s


In [15]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-01 00:00:31,596,2144,66.50,31,0,0,0,1,...,1.0,66.500000,1.0,66.500000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-04-01 00:01:59,3305,7837,75.42,119,0,0,0,1,...,1.0,75.420000,1.0,75.420000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-04-01 00:03:12,53,7674,15.11,192,0,0,0,1,...,1.0,15.110000,1.0,15.110000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-04-01 00:04:03,1311,3615,122.18,243,0,0,0,1,...,1.0,122.180000,1.0,122.180000,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-04-01 00:06:20,4459,169,60.67,380,0,0,0,1,...,1.0,60.670000,1.0,60.670000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786024,1786024,2018-09-30 23:57:44,1963,9225,19.58,15811064,182,0,0,1,...,32.0,92.100938,122.0,89.237049,2.0,0.0,6.0,0.0,35.0,0.0
1786025,1786025,2018-09-30 23:58:24,2878,4178,56.47,15811104,182,0,0,1,...,23.0,52.699130,103.0,54.361942,1.0,0.0,11.0,0.0,38.0,0.0
1786026,1786026,2018-09-30 23:58:26,3630,8878,12.55,15811106,182,0,0,1,...,17.0,59.020000,67.0,59.234627,0.0,0.0,8.0,0.0,30.0,0.0
1786027,1786027,2018-09-30 23:59:31,3531,192,101.36,15811171,182,0,0,1,...,13.0,58.278462,71.0,56.860986,1.0,0.0,9.0,0.0,26.0,0.0


In [16]:
#Terminal ID transformations

In [18]:
#NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
#NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY

In [19]:
#RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW

In [20]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_DATETIME')
    
    terminal_transactions.index=terminal_transactions.TX_DATETIME
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions

In [21]:
transactions_df[transactions_df.TX_FRAUD==1]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
932,932,2018-04-01 05:11:06,3744,4065,280.33,18666,0,1,1,1,...,2.0,228.880000,2.0,228.880000,0.0,0.0,0.0,0.000000,0.0,0.000000
1628,1628,2018-04-01 06:47:47,495,9609,223.10,24467,0,1,1,1,...,2.0,173.230000,2.0,173.230000,0.0,0.0,0.0,0.000000,0.0,0.000000
2393,2393,2018-04-01 08:16:11,1241,9394,47.79,29771,0,1,2,1,...,1.0,47.790000,1.0,47.790000,0.0,0.0,0.0,0.000000,0.0,0.000000
3054,3054,2018-04-01 09:26:30,4644,3372,220.94,33990,0,1,1,1,...,2.0,154.830000,2.0,154.830000,0.0,0.0,0.0,0.000000,0.0,0.000000
4778,4778,2018-04-01 11:56:11,2611,6872,238.70,42971,0,1,1,1,...,3.0,139.800000,3.0,139.800000,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785809,1785809,2018-09-30 22:06:21,1701,4515,89.78,15804381,182,1,2,1,...,10.0,62.699000,56.0,55.453393,0.0,0.0,3.0,1.000000,21.0,0.190476
1785820,1785820,2018-09-30 22:11:09,4547,8673,26.02,15804669,182,1,2,1,...,23.0,36.667391,104.0,37.062596,1.0,1.0,7.0,0.714286,32.0,0.156250
1785850,1785850,2018-09-30 22:29:02,1265,9024,23.60,15805742,182,1,3,1,...,10.0,91.259000,52.0,44.939808,2.0,0.0,10.0,0.000000,42.0,0.000000
1785879,1785879,2018-09-30 22:39:40,349,6309,220.76,15806380,182,1,1,1,...,17.0,95.211176,93.0,100.149785,0.0,0.0,9.0,0.000000,37.0,0.000000


In [22]:
#Let us compute these six features for the first terminal ID containing at least one fraud:
# Get the first terminal ID that contains frauds
transactions_df[transactions_df.TX_FRAUD==0].TERMINAL_ID[0]

2144

In [23]:
get_count_risk_rolling_window(transactions_df[transactions_df.TERMINAL_ID==3059], delay_period=7, windows_size_in_days=[1,7,30])

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,,,,,
5358,5358,2018-04-01 12:47:02,1281,3059,96.23,46022,0,0,0,1,...,2.0,79.085000,2.0,79.085000,0.0,0.0,0.0,0.0,0.0,0.0
8091,8091,2018-04-01 17:14:18,4120,3059,48.46,62058,0,0,0,1,...,4.0,34.020000,4.0,34.020000,0.0,0.0,0.0,0.0,0.0,0.0
24323,24323,2018-04-03 11:48:25,1011,3059,46.54,215305,2,0,0,0,...,5.0,33.688000,5.0,33.688000,0.0,0.0,0.0,0.0,0.0,0.0
34190,34190,2018-04-04 11:56:42,3877,3059,49.39,302202,3,0,0,0,...,6.0,73.508333,6.0,73.508333,0.0,0.0,0.0,0.0,0.0,0.0
37316,37316,2018-04-04 16:50:40,1205,3059,105.83,319840,3,0,0,0,...,8.0,82.606250,8.0,82.606250,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733816,1733816,2018-09-25 14:27:22,2988,3059,89.70,15344842,177,0,0,0,...,14.0,53.070714,91.0,48.858242,0.0,0.0,6.0,0.0,26.0,0.0
1754326,1754326,2018-09-27 16:14:23,2275,3059,161.45,15524063,179,0,0,0,...,14.0,72.664286,66.0,79.673485,2.0,0.0,8.0,0.0,28.0,0.0
1756016,1756016,2018-09-27 20:47:46,1469,3059,19.49,15540466,179,0,0,0,...,35.0,19.834857,102.0,19.892941,2.0,0.0,9.0,0.0,29.0,0.0


In [24]:
#Let us finally generate these features for all terminals
%time transactions_df=transactions_df.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

Wall time: 1min 41s


In [25]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-01 00:00:31,596,2144,66.50,31,0,0,0,1,...,1.0,66.500000,1.0,66.500000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-04-01 00:01:59,3305,7837,75.42,119,0,0,0,1,...,1.0,75.420000,1.0,75.420000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-04-01 00:03:12,53,7674,15.11,192,0,0,0,1,...,1.0,15.110000,1.0,15.110000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-04-01 00:04:03,1311,3615,122.18,243,0,0,0,1,...,1.0,122.180000,1.0,122.180000,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-04-01 00:06:20,4459,169,60.67,380,0,0,0,1,...,1.0,60.670000,1.0,60.670000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786024,1786024,2018-09-30 23:57:44,1963,9225,19.58,15811064,182,0,0,1,...,32.0,92.100938,122.0,89.237049,2.0,0.0,6.0,0.0,35.0,0.0
1786025,1786025,2018-09-30 23:58:24,2878,4178,56.47,15811104,182,0,0,1,...,23.0,52.699130,103.0,54.361942,1.0,0.0,11.0,0.0,38.0,0.0
1786026,1786026,2018-09-30 23:58:26,3630,8878,12.55,15811106,182,0,0,1,...,17.0,59.020000,67.0,59.234627,0.0,0.0,8.0,0.0,30.0,0.0
1786027,1786027,2018-09-30 23:59:31,3531,192,101.36,15811171,182,0,0,1,...,13.0,58.278462,71.0,56.860986,1.0,0.0,9.0,0.0,26.0,0.0


In [26]:
#Saving of dataset

In [27]:
DIR_OUTPUT = "./simulated-data-transformed/data/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.datetime.strptime("2018-04-01", "%Y-%m-%d")

for day in range(transactions_df.TX_TIME_DAYS.max()+1):
    
    transactions_day = transactions_df[transactions_df.TX_TIME_DAYS==day].sort_values('TX_TIME_SECONDS')
    
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    # Protocol=4 required for Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)